In [1]:
#%%
print("ok")

ok


In [2]:
allNormalText = open(r"./data/data-normal.txt", 'r', encoding='utf-8').read()
allNormalTextArray = allNormalText.split('\n')
print(len(allNormalTextArray))

allAdText = open(r"./data/data-pssisterad.txt", 'r', encoding='utf-8').read()
allAdTextArray = allAdText.split('\n')
print(len(allAdTextArray))

185
72


In [3]:
from gensim.models import Word2Vec
from model import AdPredictor
adp = AdPredictor()
splitNormalWords = [list(adp.splitWords(ad, False)) for ad in allNormalTextArray+allAdTextArray]
print(len(splitNormalWords))
splitNormalWords[0:10]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kenvix\AppData\Local\Temp\jieba.cache
Loading model cost 0.760 seconds.
Prefix dict has been built successfully.
257


[['没有', '上次', '学长', '学姐', '教', '几节课'],
 ['学姐', '交', '一下'],
 ['认识', '学姐', '保研'],
 ['党明', '学姐', '真的', '强'],
 ['想不到', '没有', '党明', '学姐', '做', '不到'],
 ['党明', '学姐'],
 ['党明', '学姐', '献上'],
 ['学姐'],
 ['我见', '证明', '党明', '学姐', '非常', '漂亮', '女孩子'],
 ['开机', '慢点', '还好']]

In [4]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dictionary = corpora.Dictionary(splitNormalWords)
new_corpus = [dictionary.doc2bow(text) for text in splitNormalWords]
print(dictionary.get("学姐"))
print(new_corpus[0:10])

None
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(2, 1), (6, 1), (7, 1)], [(2, 1), (8, 1), (9, 1)], [(2, 1), (10, 1), (11, 1), (12, 1)], [(2, 1), (5, 1), (10, 1), (13, 1), (14, 1), (15, 1)], [(2, 1), (10, 1)], [(2, 1), (10, 1), (16, 1)], [(2, 1)], [(2, 1), (10, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1)]]


In [23]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("tfidf.gsm")

# 使用这个训练好的模型得到单词的tfidf值
tfidf_vec = []
tfidf_vocabs = []
for i in range(len(splitNormalWords)):
    string_bow = dictionary.doc2bow(splitNormalWords[i])
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)

print(tfidf_vec[0:5])
tfidf_vocabs_word_to_id = dictionary.token2id
print(tfidf_vocabs_word_to_id.get("学姐"))
print(tfidf_vocabs_word_to_id.get("大佬"))
# print(tfidf_vocabs)
tfidf_vocabs = {value:key for (key, value) in tfidf_vocabs_word_to_id.items()}
print(tfidf_vocabs[0])

[[(0, 0.4602581128821097), (1, 0.4602581128821097), (2, 0.23564377058789043), (3, 0.4602581128821097), (4, 0.4602581128821097), (5, 0.31164385360202146)], [(2, 0.42981837308890347), (6, 0.5249204813955685), (7, 0.7346527440680071)], [(2, 0.3595264199034269), (8, 0.7022250201784704), (9, 0.6145085633469818)], [(2, 0.3217561308178781), (10, 0.44617781992931005), (11, 0.6284522998805557), (12, 0.5499509542305356)], [(2, 0.29056380361078027), (5, 0.3842767549025883), (10, 0.40292355616001396), (13, 0.36851110907890555), (14, 0.3842767549025883), (15, 0.56752761843906)]]
2
153
上次


In [81]:
from collections import OrderedDict
import numpy as np

def tfidf_wtd_avg_word_vectors(words,tfidf_vector,tfidf_vocabulary,model,num_features):
    #print("tfidf_vector", tfidf_vector)
    word_tfidfs = []
    word_tfidf_map = OrderedDict()

    for (key, proba) in tfidf_vector:
        word_tfidfs.append(proba)
        word_tfidf_map[tfidf_vocabulary[key]] = proba
    
    # print(word_tfidfs)
    # print(word_tfidf_map)
    
    feature_vector=np.zeros((num_features,),dtype='float64')
    vocabulary=set(model.wv.index2word)
    wts=0
    for word in words:
        if word in vocabulary:
            word_vector=model[word]
            weighted_word_vector=word_tfidf_map[word]*word_vector
            wts=wts+word_tfidf_map[word]
            feature_vector=np.add(feature_vector,weighted_word_vector)
    if wts:
        feature_vector=np.divide(feature_vector,wts)
    return feature_vector

def tfidf_weighted_averaged_word_vectorizer(corpus,tfidf_vectors,tfidf_vocabulary,model,num_features):
    docs_tfidfs=[(doc,doc_tfidf) for doc,doc_tfidf in zip(corpus,tfidf_vectors)]
    features=[tfidf_wtd_avg_word_vectors(tokenized_sentence,tfidf,tfidf_vocabulary,model,num_features) for tokenized_sentence,tfidf in docs_tfidfs]
    return np.array(features)

# vec = Word2Vec(splitNormalWords,size=10,window=10,min_count=2,sample=1e-3)
word2vec = Word2Vec(splitNormalWords, min_count=3, sample=1e-3, size=10, window=10)
vector = tfidf_weighted_averaged_word_vectorizer(corpus=splitNormalWords, tfidf_vectors=tfidf_vec, tfidf_vocabulary=tfidf_vocabs,model=word2vec,num_features=10)
vector = np.array(vector)
# vec.build_vocab(splitNormalWords)
# vec.train(splitNormalWords, total_examples=len(splitNormalWords), epochs=vec.epochs)
# # vec.save("vec.gsm")
# vec
print(len(vector))
word2vec.save("word2vec.gsm")
vector[0:100]

257


02,  3.94364332e-03,
        -2.35571848e-02, -1.75726012e-02,  2.17364642e-03,
        -2.51707859e-02],
       [-2.75715130e-02, -1.60476701e-03, -4.39499937e-02,
         7.87939500e-03,  1.18300632e-02, -6.92914276e-03,
        -2.55796808e-02,  1.09308553e-03,  1.47001833e-02,
        -2.90677029e-02],
       [-2.48834897e-02, -1.43868237e-02, -4.02918613e-03,
         1.89338568e-02,  1.31068082e-02, -5.44056862e-03,
        -1.00856926e-02, -5.98123942e-03, -5.97891209e-03,
         5.55390323e-03],
       [-5.85340248e-03,  8.95757513e-03, -1.32620822e-02,
         5.96175297e-03,  2.70497179e-02, -1.20213917e-02,
        -1.93209308e-02, -3.14607639e-04, -4.72602641e-03,
         6.92828413e-03],
       [-2.48834911e-02, -1.43868239e-02, -4.02918636e-03,
         1.89338580e-02,  1.31068087e-02, -5.44056893e-03,
        -1.00856934e-02, -5.98123965e-03, -5.97891160e-03,
         5.55390369e-03],
       [-2.48834912e-02, -1.43868245e-02, -4.02918665e-03,
         1.89338572e-02

In [116]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

_totalNormalSize = len(allNormalTextArray)
_totalAdSize = len(allAdTextArray)
actualNormalSize = _totalNormalSize
actualAdSize = _totalAdSize

y = np.concatenate((np.zeros(_totalNormalSize), np.ones(_totalAdSize)), axis=0)
#y = (np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray)))

_i = 0
docVector = []

for v in vector:
    mean = np.mean(v)

    if mean < 0.00000001:
        if _i >= _totalNormalSize:
            actualAdSize -= 1
        else:
            actualNormalSize -= 1
    else:
        docVector.append(v)

    _i += 1

print(len(vector))
print(_totalNormalSize, _totalAdSize)
print(actualNormalSize, actualAdSize)
print(len(docVector))

257
185 72
54 21
75


In [128]:
y = np.concatenate((np.zeros(_totalNormalSize), np.ones(_totalAdSize)), axis=0)
#y = (np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray)))

X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.10, train_size=0.90, random_state=100)

svmModel = SVC()  # 使用RBF核
svmModelRes = svmModel.fit(X_train, y_train)
print(svmModelRes.score(X_test, y_test))

0.7692307692307693


In [22]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

# y = trainer.train_model()
y = np.concatenate((np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray))), axis=0)

print(len(y))
print(len(vector))

clf = MultinomialNB(alpha=1, fit_prior=True)
clf.fit(vector, y)

result = pd.DataFrame(vector)
keys = []
values = []

for key, value in vector:
    keys.append(key)
    values.append(value)

df = pd.DataFrame(data={"key": keys, "value": values})
colnames = df.sort_values("value")["key"].values
result.columns = colnames

#X_train, X_test, y_train, y_test = train_test_split(textMatrix, y, test_size=0.10, random_state=100)
print("Test Acc", clf.score(X_test, y_test))

257
257


ValueError: Negative values in data passed to MultinomialNB (input X)

In [57]:
import sklearn
sklearn.__version__

'0.22.1'